## Micropython memory map visualizer

The below class is used to visualize the memory map of MicroPython. It is based on the `micropython.memory_info(1)` output.  
It can be used to visualize the current usage of the memory befor and after gc collection, or to visualize the difference between two memory maps.

_Note:_ Currently the Memoryinfo class is included in this notebook, but the intent is for it to be moved into the magic module in the future.


In [12]:
from micropython_magic.memoryinfo import MemoryInfo

In [13]:
# issue soft reset before running
%mpy --soft-reset
mem_lst = %mpy import micropython; micropython.mem_info(True)
mem_lst

True


['stack: 500 out of 7936',
 'GC: total: 192064, used: 4480, free: 187584',
 ' No. of 1-blocks: 52, 2-blocks: 14, max blk sz: 64, max free sz: 11714',
 'GC memory layout; from 20009520:',
 '00000000: h=MhhhBD.hBTTBDBDBhTh===BTB=Dh====B=BBBBBBTB=BTB=BBBTB=TBTB=Bh==',
 '00000400: =DB=h===========h===================BBBh==h=h=====h=============',
 '00000800: ====h===========================================================',
 '00000c00: ====h===========================================================',
 '00001000: ====h=hhhBhhhhBh=h..h..h=.....h=h=..............................',
 '       (182 lines all free)',
 '0002ec00: ....................................']

In [14]:
# create the memory map using the output, and display it
# map_1 = MemoryInfo(mem_lst.data, columns=1)
map_1 = MemoryInfo(mem_lst, cols=1)
map_1

Stack used:  0x01f4 of Total: 0x1f00  pct free: 93.7%
Memory used: 0x000_1180 of Total: 0x002_ee40  free: 0x002_dcc0 pct free: 97.7%
1-Blocks:          52  2-Blocks:         14
Max Block size:    64  Max Free size: 11714
h=MhhhBD.hBTTBDBDBhTh===BTB=Dh====B=BBBBBBTB=BTB=BBBTB=TBTB=Bh== 
=DB=h===========h===================BBBh==h=h=====h============= 
====h=========================================================== 
====h=========================================================== 
====h=hhhBhhhhBh=h..h..h=.....h=h=.............................. 
....................................

In [15]:
# default = 4 columns
map_1.columns = 2
map_1

Stack used:  0x01f4 of Total: 0x1f00  pct free: 93.7%
Memory used: 0x000_1180 of Total: 0x002_ee40  free: 0x002_dcc0 pct free: 97.7%
1-Blocks:          52  2-Blocks:         14
Max Block size:    64  Max Free size: 11714
h=MhhhBD.hBTTBDBDBhTh===BTB=Dh====B=BBBBBBTB=BTB=BBBTB=TBTB=Bh== =DB=h===========h===================BBBh==h=h=====h============= 
====h=========================================================== ====h=========================================================== 
====h=hhhBhhhhBh=h..h..h=.....h=h=.............................. ....................................

In [16]:
map_1.rainbow = True  # alternating colors for each block
map_1

Stack used:  0x01f4 of Total: 0x1f00  pct free: 93.7%
Memory used: 0x000_1180 of Total: 0x002_ee40  free: 0x002_dcc0 pct free: 97.7%
1-Blocks:          52  2-Blocks:         14
Max Block size:    64  Max Free size: 11714
h=MhhhBD.hBTTBDBDBhTh===BTB=Dh====B=BBBBBBTB=BTB=BBBTB=TBTB=Bh== =DB=h===========h===================BBBh==h=h=====h============= 
====h=========================================================== ====h=========================================================== 
====h=hhhBhhhhBh=h..h..h=.....h=h=.............................. ....................................

In [17]:
# consume some more memory
%mpy print(help('modules'))
%mpy import gc; gc.collect()

mem_lst = %mpy import micropython; micropython.mem_info(True)
map_2=MemoryInfo(mem_lst, cols=2 )
map_2.rainbow = True
map_2

Stack used:  0x01f4 of Total: 0x1f00  pct free: 93.7%
Memory used: 0x000_1070 of Total: 0x002_ee40  free: 0x002_ddd0 pct free: 97.8%
1-Blocks:          35  2-Blocks:         14
Max Block size:    64  Max Free size: 11634
h=Mhhh=DhhhhhBD.DBh.h===B.B=Dh====B=BBBBBB.B=B.B=BBB.B=.B.B=Bh== =DB=h===========h===================BBBh==h=h=====h============= 
====h=========================================================== ====h=========================================================== 
====...........h=h=..h=......................................... .............................hh.B...............h=.............. 
....................................

In [18]:
# %%micropython
# eat even more memory - with disabled gc
import gc

gc.disable()
foo = "Parrot"
try:
    for _ in range(15):
        foo = foo + foo
except MemoryError as e:
    print(e)
    pass

memory allocation failed, allocating 98305 bytes


In [19]:
# take anoter memory snapshot 
mem_lst = %mpy micropython.mem_info(True)
map_3=MemoryInfo(mem_lst)
map_3

Stack used:  0x01f4 of Total: 0x1f00  pct free: 93.7%
Memory used: 0x001_a6c0 of Total: 0x002_ee40  free: 0x001_4780 pct free: 43.7%
1-Blocks:         124  2-Blocks:         25
Max Block size:  3073  Max Free size:  5231
h=Mhhh=DhhhhhBDhDBhhh===BhB=Dh====B=BBBBBBhB=BhB=BBBBB=hBhB=Bh== =DB=h===========h===================BBBhhhh=h=====h============= ====h=========================================================== ====h=========================================================== 
====hBhhhhhh===h=h=h=h=hhh=======h========T==Shh=h===hhBhhhhBhhh hhhh=T==hhh=h===Shh===hhBShhhhhhBhSh=Bhhhhh=Bhh=h=h===h========= ==============================================================hh BShhBhhh=SSShSh=hhhhhSh===h=============B....h=h===SSh=====Sh=== 
==========h===================================================== ==================h====h======h============h==================== ====h================================================h========== ==============================================================

In [20]:
map_3.rainbow = True
map_3

Stack used:  0x01f4 of Total: 0x1f00  pct free: 93.7%
Memory used: 0x001_a6c0 of Total: 0x002_ee40  free: 0x001_4780 pct free: 43.7%
1-Blocks:         124  2-Blocks:         25
Max Block size:  3073  Max Free size:  5231
h=Mhhh=DhhhhhBDhDBhhh===BhB=Dh====B=BBBBBBhB=BhB=BBBBB=hBhB=Bh== =DB=h===========h===================BBBhhhh=h=====h============= ====h=========================================================== ====h=========================================================== 
====hBhhhhhh===h=h=h=h=hhh=======h========T==Shh=h===hhBhhhhBhhh hhhh=T==hhh=h===Shh===hhBShhhhhhBhSh=Bhhhhh=Bhh=h=h===h========= ==============================================================hh BShhBhhh=SSShSh=hhhhhSh===h=============B....h=h===SSh=====Sh=== 
==========h===================================================== ==================h====h======h============h==================== ====h================================================h========== ==============================================================

In [21]:
# compare snapshot 1 and 2 to see
#  - what remained the same = Black on Blue
#  - what was freed up = Green on Green ( can also be due to additional pages being reported)
#  - what was allocated = Yellow or White on Red

map_3 - map_1

 --> 
Stack used:  0x01f4 of Total: 0x1f00  pct free: 93.7%
Memory used: 0x001_a6c0 of Total: 0x002_ee40  free: 0x001_4780 pct free: 43.7%
1-Blocks:         124  2-Blocks:         25
Max Block size:  3073  Max Free size:  5231
h=Mhhh=DhhhhhBDhDBhhh===BhB=Dh====B=BBBBBBhB=BhB=BBBBB=hBhB=Bh== =DB=h===========h===================BBBhhhh=h=====h============= ====h=========================================================== ====h=========================================================== 
====hBhhhhhh===h=h=h=h=hhh=======h========T==Shh=h===hhBhhhhBhhh hhhh=T==hhh=h===Shh===hhBShhhhhhBhSh=Bhhhhh=Bhh=h=h===h========= ==============================================================hh BShhBhhh=SSShSh=hhhhhSh===h=============B....h=h===SSh=====Sh=== 
==========h===================================================== ==================h====h======h============h==================== ====h================================================h========== ========================================================

In [23]:
# now free up some memory and take another memory snapshot 
%mpy import gc; gc.collect()
mem_lst = %mpy import micropython; micropython.mem_info(True)
map_4=MemoryInfo(mem_lst )
map_4.rainbow = True
map_4.show_free = True
map_4

Stack used:  0x01f4 of Total: 0x1f00  pct free: 93.7%
Memory used: 0x000_d660 of Total: 0x002_ee40  free: 0x002_17e0 pct free: 71.4%
1-Blocks:          37  2-Blocks:         14
Max Block size:  3073  Max Free size:  5238
h=Mhhh=DhhhhhBDhDBhhh===B.B=Dh====B=BBBBBB.B=BBB=BBB.B=.B.B=Bh== =DB=h===========h===================BBBh..h=h=====h============= ====h=========================================================== ====h=========================================================== 
====........h=.h=..h=..h=h=======h========...................... ......................................................h========= ==============================================================.. .............................h================================== 
================================================================ ================================================================ ================================================================ ==============================================================

In [24]:
# and show the difference between the last two snapshots
map_4 - map_3

 --> 
Stack used:  0x01f4 of Total: 0x1f00  pct free: 93.7%
Memory used: 0x000_d660 of Total: 0x002_ee40  free: 0x002_17e0 pct free: 71.4%
1-Blocks:          37  2-Blocks:         14
Max Block size:  3073  Max Free size:  5238
h=Mhhh=DhhhhhBDhDBhhh===B.B=Dh====B=BBBBBB.B=BBB=BBB.B=.B.B=Bh== =DB=h===========h===================BBBh..h=h=====h============= ====h=========================================================== ====h=========================================================== 
====........h=.h=..h=..h=h=======h========...................... ......................................................h========= ==============================================================.. .............................h================================== 
================================================================ ================================================================ ================================================================ ========================================================

In [25]:
# or the reverse to see what got freed ( on RED)
map_3 - map_4

 --> 
Stack used:  0x01f4 of Total: 0x1f00  pct free: 93.7%
Memory used: 0x001_a6c0 of Total: 0x002_ee40  free: 0x001_4780 pct free: 43.7%
1-Blocks:         124  2-Blocks:         25
Max Block size:  3073  Max Free size:  5231
h=Mhhh=DhhhhhBDhDBhhh===BhB=Dh====B=BBBBBBhB=BhB=BBBBB=hBhB=Bh== =DB=h===========h===================BBBhhhh=h=====h============= ====h=========================================================== ====h=========================================================== 
====hBhhhhhh===h=h=h=h=hhh=======h========T==Shh=h===hhBhhhhBhhh hhhh=T==hhh=h===Shh===hhBShhhhhhBhSh=Bhhhhh=Bhh=h=h===h========= ==============================================================hh BShhBhhh=SSShSh=hhhhhSh===h=============B....h=h===SSh=====Sh=== 
==========h===================================================== ==================h====h======h============h==================== ====h================================================h========== ========================================================

In [26]:
# are we back to where we started?
map_4 - map_1

 --> 
Stack used:  0x01f4 of Total: 0x1f00  pct free: 93.7%
Memory used: 0x000_d660 of Total: 0x002_ee40  free: 0x002_17e0 pct free: 71.4%
1-Blocks:          37  2-Blocks:         14
Max Block size:  3073  Max Free size:  5238
h=Mhhh=DhhhhhBDhDBhhh===B.B=Dh====B=BBBBBB.B=BBB=BBB.B=.B.B=Bh== =DB=h===========h===================BBBh..h=h=====h============= ====h=========================================================== ====h=========================================================== 
====........h=.h=..h=..h=h=======h========...................... ......................................................h========= ==============================================================.. .............................h================================== 
================================================================ ================================================================ ================================================================ ========================================================

In [27]:
%mpy print(dir())

["['machine', 'os', 'gc', 'f', '__name__', 'rp2', 'w', 'foo', '_', 'micropython']"]

In [30]:
%mpy del foo, f, w
%mpy import gc; gc.collect()

# take another memory snapshot 
mem_lst = %mpy import micropython; micropython.mem_info(True)
map_5=MemoryInfo(mem_lst )

map_5-map_1

 --> 
Stack used:  0x01f4 of Total: 0x1f00  pct free: 93.7%
Memory used: 0x000_11b0 of Total: 0x002_ee40  free: 0x002_dc90 pct free: 97.6%
1-Blocks:          35  2-Blocks:         14
Max Block size:    64  Max Free size:  6458
h=Mhhh=DhhhhhBD.DBh.h===B.B=Dh====B=BBBBBB.B=B.B=BBB.B=.B.B=Bh== =DB=h===========h===================BBB...h=h=====h============= ====h=========================================================== ====h=========================================================== 
====h.h.B..h=..h=.....h=.h=======h========..h=.................. ........................................h=====.................. ....................................

In [31]:
map_5 - map_4

 --> 
Stack used:  0x01f4 of Total: 0x1f00  pct free: 93.7%
Memory used: 0x000_11b0 of Total: 0x002_ee40  free: 0x002_dc90 pct free: 97.6%
1-Blocks:          35  2-Blocks:         14
Max Block size:    64  Max Free size:  6458
h=Mhhh=DhhhhhBD.DBh.h===B.B=Dh====B=BBBBBB.B=B.B=BBB.B=.B.B=Bh== =DB=h===========h===================BBB...h=h=====h============= ====h=========================================================== ====h=========================================================== 
====h.h.B..h=..h=.....h=.h=======h========..h=.................. ........................................h=====.................. ....................................

In [32]:
map_5

 --> 
Stack used:  0x01f4 of Total: 0x1f00  pct free: 93.7%
Memory used: 0x000_11b0 of Total: 0x002_ee40  free: 0x002_dc90 pct free: 97.6%
1-Blocks:          35  2-Blocks:         14
Max Block size:    64  Max Free size:  6458
h=Mhhh=DhhhhhBD.DBh.h===B.B=Dh====B=BBBBBB.B=B.B=BBB.B=.B.B=Bh== =DB=h===========h===================BBB...h=h=====h============= ====h=========================================================== ====h=========================================================== 
====h.h.B..h=..h=.....h=.h=======h========..h=.................. ........................................h=====.................. ....................................